## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Casablanca,33.5928,-7.6192,62.94,93,40,4.00,MA,scattered clouds
1,1,Severo-Kurilsk,50.6789,156.1250,37.56,91,93,9.08,RU,overcast clouds
2,2,East London,-33.0153,27.9116,69.75,86,100,24.45,ZA,overcast clouds
3,3,Faanui,-16.4833,-151.7500,79.29,69,14,11.05,PF,few clouds
4,4,Mudanya,40.3753,28.8822,49.37,59,4,2.86,TR,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Faanui,-16.4833,-151.7500,79.29,69,14,11.05,PF,few clouds
5,5,Caruray,10.3094,119.0128,78.66,86,66,3.60,PH,broken clouds
6,6,Puerto Leguizamo,-0.1934,-74.7819,77.77,91,95,2.46,CO,overcast clouds
7,7,Atuona,-9.8000,-139.0333,78.21,75,99,12.57,PF,overcast clouds
8,8,Georgetown,5.4112,100.3354,78.89,94,40,1.99,MY,scattered clouds
9,9,Rikitea,-23.1203,-134.9692,73.40,65,0,6.08,PF,clear sky
14,14,Arraial Do Cabo,-22.9661,-42.0278,73.36,53,20,8.05,BR,few clouds
23,23,Seybaplaya,19.6500,-90.6667,77.74,83,80,7.02,MX,broken clouds
27,27,Mackay,-21.1500,149.2000,75.69,94,43,2.30,AU,scattered clouds
31,31,Sarankhola,22.3082,89.7897,70.50,76,17,3.56,BD,few clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Country                209
Current Description    211
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                209
City                   209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Country                209
Current Description    209
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,79.29,few clouds,-16.4833,-151.7500,
5,Caruray,PH,78.66,broken clouds,10.3094,119.0128,
6,Puerto Leguizamo,CO,77.77,overcast clouds,-0.1934,-74.7819,
7,Atuona,PF,78.21,overcast clouds,-9.8000,-139.0333,
8,Georgetown,MY,78.89,scattered clouds,5.4112,100.3354,
9,Rikitea,PF,73.40,clear sky,-23.1203,-134.9692,
14,Arraial Do Cabo,BR,73.36,few clouds,-22.9661,-42.0278,
23,Seybaplaya,MX,77.74,broken clouds,19.6500,-90.6667,
27,Mackay,AU,75.69,scattered clouds,-21.1500,149.2000,
31,Sarankhola,BD,70.50,few clouds,22.3082,89.7897,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
cancel = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = cancel.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,79.29,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
6,Puerto Leguizamo,CO,77.77,overcast clouds,-0.1934,-74.7819,la casona de juancho
7,Atuona,PF,78.21,overcast clouds,-9.8000,-139.0333,Villa Enata
8,Georgetown,MY,78.89,scattered clouds,5.4112,100.3354,Cititel Penang
9,Rikitea,PF,73.40,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
552,Hambantota,LK,75.42,broken clouds,6.1241,81.1185,Bungalow 63
556,Cravinhos,BR,86.72,broken clouds,-21.3403,-47.7294,Ônix Inn Hotel Cravinhos
557,San Quintin,MX,70.74,overcast clouds,30.4833,-115.9500,Don Eddie's Sport Fishing Center
559,San Patricio,US,85.01,scattered clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"


In [27]:
# 8a. Create the output File (CSV)
output_data_file = 'weatherpy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))